In [ ]:
#%matplotlib notebook
%matplotlib inline
#%config InlineBackend.figure_formats=['svg']
#%config InlineBackend.figure_formats=['pdf']

import matplotlib
matplotlib.rcParams['figure.figsize'] = (12, 9)

from matplotlib.colors import LogNorm
from matplotlib.ticker import FuncFormatter

import scipy
import scipy.stats
import math
import numpy as np
import pandas as pd

#import ipywidgets
#from ipywidgets import interact

## Reminder of last results

### Improvement ratio for $\Delta\psi$ < 0.2°

The conclusion of the last weekly meeting:
* with the wavelet configuration at used that time,
* with cuts at 50 NPE on cleaned image,
* the gain ratio of wavelets / tailcut  was almost 0% (with a small population of ~350 images)

### What happen if we change the requested precision angle ?

See http://localhost:8888/notebooks/first_bin_delta_psi_func.ipynb

CCL:
* The ratio depends a lot on this requested precision angle.
* Despite the low statistics, a similar pattern is observed with other wavelet configurations.
* Systematics ?
* Why ?

### New configuration: reduce the threshold for small scales

During my postdoc seminar, I presented several new configurations for wavelets.
One of them gives escpcially good results on the $\Delta\psi$ estimator:
* mr_filter -n4 -K -k -C1 -s2,2,3,3 -m3 --kill-isolated-pixels
* in this configuration, the threshold is reduced for small scales only

#### PE sum

##### Initial configuration (-n4 -K -k -C1 -s3 -m3  --kill-isolated-pixels)

See http://localhost:8888/notebooks/npe_analysis-TMP-former-ref.ipynb#

##### New configuration (-n4 -K -k -C1 -s2,2,3,3 -m3  --kill-isolated-pixels)

See http://localhost:8888/notebooks/npe_analysis.ipynb

#### $\Delta\psi$ histogram for bright and faint showers on images

In [ ]:
# Plot delta psi #####################

def plot_ratio(ax, res_tuple_tc, res_tuple_wt):
    val_of_bins_tc, bins_tc, patches_tc = res_tuple_tc
    val_of_bins_wt, bins_wt, patches_wt = res_tuple_wt
    edges_of_bins = bins_tc

    # Set ratio where val_of_bins_data is not zero
    ratio = np.divide(val_of_bins_wt,
                      val_of_bins_tc,
                      where=(val_of_bins_tc != 0))

    # Compute error on ratio (null if cannot be computed)
    # This is wrong as it's made for Gaussian distributions and here we have Poisson distribution
    error = np.divide(val_of_bins_wt * np.sqrt(val_of_bins_tc) + val_of_bins_tc * np.sqrt(val_of_bins_wt),
                       np.power(val_of_bins_tc, 2),
                       where=(val_of_bins_tc != 0))

    ax.set_ylabel('Ratio (WT/TC)', fontsize=20)
    ax.axhline(y=1, linewidth=2, linestyle='--', color='gray', alpha=0.5)

    bincenter = 0.5 * (edges_of_bins[1:] + edges_of_bins[:-1])
    ax.errorbar(bincenter, ratio, yerr=error, fmt='o', color='k', elinewidth=3, capsize=4, capthick=3, linewidth=6)
    ax.plot(bincenter, ratio, 'ok', linewidth=6)

################################

def plot_delta_psi(df):
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(16, 9))

    #BINS_START, BINS_STOP, BINS_STEP = 0., 99., 9.
    #BINS_START, BINS_STOP, BINS_STEP = 0., 99., 9.

    #BINS = np.arange(BINS_START, BINS_STOP, BINS_STEP)
    #BINS = 30
    #BINS = np.linspace(BINS_START, BINS_STOP, 30)
    #BINS = [3.53056780e-03, 5.99515231e+00, 1.19867741e+01, 1.79783958e+01, 2.39700176e+01, 2.99616393e+01,
    #        3.59532610e+01, 4.19448828e+01, 4.79365045e+01, 5.39281263e+01, 5.99197480e+01, 6.59113698e+01,
    #        7.19029915e+01, 7.78946133e+01, 8.38862350e+01, 8.98778568e+01]
    BINS = 15

    ################################

    NPE_MIN, NPE_MAX = 50, FAINT_BRIGHT_BORDER

    df1 = df[df.peSum_tc > NPE_MIN][df.peSum_tc <= NPE_MAX]['delta_tc']
    df2 = df[df.peSum_wt > NPE_MIN][df.peSum_wt <= NPE_MAX]['delta_wt']

    res_tuple_tc1 = ax1.hist(df1,
                             label="TC num={}".format(len(df1)),
                             bins=BINS,
                             linewidth=2, alpha=.5, color='blue', histtype="step")

    res_tuple_wt1 = ax1.hist(df2,
                             label="WT num={}".format(len(df2)),
                             bins=BINS,
                             linewidth=2, alpha=.5, color='red',  histtype="step")

    ax1.set_title("{} to {} NPE".format(NPE_MIN, NPE_MAX), fontsize=20)
    ax1.set_xlabel("delta_psi", fontsize=20)
    ax1.set_ylabel("counts", fontsize=20)
    ax1.legend(prop={'size': 18}, loc='best', fancybox=True, framealpha=0.5)

    plot_ratio(ax3, res_tuple_tc1, res_tuple_wt1)

    ################################

    NPE_MIN, NPE_MAX = FAINT_BRIGHT_BORDER, 2000

    df1 = df[df.peSum_tc > NPE_MIN][df.peSum_tc <= NPE_MAX]['delta_tc']
    df2 = df[df.peSum_wt > NPE_MIN][df.peSum_wt <= NPE_MAX]['delta_wt']

    res_tuple_tc2 = ax2.hist(df1,
                             label="TC num={}".format(len(df1)),
                             bins=BINS,
                             linewidth=2, alpha=.5, color='blue', histtype="step")

    res_tuple_wt2 = ax2.hist(df2,
                             label="WT num={}".format(len(df2)),
                             bins=BINS,
                             linewidth=2, alpha=.5, color='red',  histtype="step")

    ax2.set_title("{} to {} NPE".format(NPE_MIN, NPE_MAX), fontsize=20)
    ax2.set_xlabel("delta_psi", fontsize=20)
    ax2.set_ylabel("counts", fontsize=20)
    ax2.legend(prop={'size': 18}, loc='best', fancybox=True, framealpha=0.5)

    plot_ratio(ax4, res_tuple_tc2, res_tuple_wt2)

    #ax.set_yscale('log')

    fig.suptitle(WAVELET_LABEL + " / " + TAILCUT_LABEL, fontsize=20)
    
    return fig

##### Initial configuration (-n4 -K -k -C1 -s3 -m3  --kill-isolated-pixels)

In [ ]:
#CSV_FILE_PATH = "../xps/best.csv"
CSV_FILE_PATH = "../xps/2017_02_20/2017_02_20.csv"

WAVELET_LABEL = "Wavelets-n4-K-k-C1-s3-m3-kill"
#WAVELET_LABEL = "WT-ref-f3-s3"
#WAVELET_LABEL = "WT-t24-f3-s4"
#WAVELET_LABEL = "WT-t28-f3-s5"
#WAVELET_LABEL = "WT-ref-s2-2-3-3"

TAILCUT_LABEL = "Tailcut-5-10-kill"

PART = 0         # 0 for gamma, 1 for protons

FAINT_BRIGHT_BORDER = 100

full_df = pd.read_csv(CSV_FILE_PATH)

ref = full_df[full_df.Type == 'Ref']
ref = ref[ref.Part == PART]

tc = full_df[full_df.Type == TAILCUT_LABEL]
tc = tc[tc.Part == PART]

wt = full_df[full_df.Type == WAVELET_LABEL]
wt = wt[wt.Part == PART]

tc_wt = pd.merge(tc, wt, on="Id", how="outer", suffixes=('_tc', '_wt'))  #.dropna(how='any')
df = pd.merge(tc_wt, ref, on="Id", how="outer")

df["delta_tc"] = np.fmod(((df['hPsi'] - df['hPsi_tc']) * 180. / np.pi), 90.)
df["delta_wt"] = np.fmod(((df['hPsi'] - df['hPsi_wt']) * 180. / np.pi), 90.)

df.delta_tc = abs(df.delta_tc)
df.delta_wt = abs(df.delta_wt)

#df = df[df.border > 0]   # CONTAINED
#df = df[df.border == 0]  # NOT CONTAINED

plot_delta_psi(df)

##### New configuration (-n4 -K -k -C1 -s2,2,3,3 -m3  --kill-isolated-pixels)

In [ ]:
CSV_FILE_PATH = "../xps/best.csv"
#CSV_FILE_PATH = "../xps/2017_02_20/2017_02_20.csv"

#WAVELET_LABEL = "Wavelets-n4-K-k-C1-s3-m3-kill"
#WAVELET_LABEL = "WT-ref-f3-s3"
#WAVELET_LABEL = "WT-t24-f3-s4"
#WAVELET_LABEL = "WT-t28-f3-s5"
WAVELET_LABEL = "WT-ref-s2-2-3-3"

TAILCUT_LABEL = "Tailcut-5-10-kill"

PART = 0         # 0 for gamma, 1 for protons

FAINT_BRIGHT_BORDER = 100

full_df = pd.read_csv(CSV_FILE_PATH)

ref = full_df[full_df.Type == 'Ref']
ref = ref[ref.Part == PART]

tc = full_df[full_df.Type == TAILCUT_LABEL]
tc = tc[tc.Part == PART]

wt = full_df[full_df.Type == WAVELET_LABEL]
wt = wt[wt.Part == PART]

tc_wt = pd.merge(tc, wt, on="Id", how="outer", suffixes=('_tc', '_wt'))  #.dropna(how='any')
df = pd.merge(tc_wt, ref, on="Id", how="outer")

df["delta_tc"] = np.fmod(((df['hPsi'] - df['hPsi_tc']) * 180. / np.pi), 90.)
df["delta_wt"] = np.fmod(((df['hPsi'] - df['hPsi_wt']) * 180. / np.pi), 90.)

df.delta_tc = abs(df.delta_tc)
df.delta_wt = abs(df.delta_wt)

#df = df[df.border > 0]   # CONTAINED
#df = df[df.border == 0]  # NOT CONTAINED

plot_delta_psi(df)

### Impact for the first bin

See: http://localhost:8888/notebooks/first_bin_delta_psi_func-TMP-new-config.ipynb

## Impact on results of showers on borders

See http://localhost:8888/notebooks/cut_on_border_analysis.ipynb

In [ ]:
CSV_FILE_PATH = "../xps/best.csv"
#CSV_FILE_PATH = "../xps/2017_02_20/2017_02_20.csv"

#WAVELET_LABEL = "Wavelets-n4-K-k-C1-s3-m3-kill"
#WAVELET_LABEL = "WT-ref-f3-s3"
#WAVELET_LABEL = "WT-t24-f3-s4"
#WAVELET_LABEL = "WT-t28-f3-s5"
WAVELET_LABEL = "WT-ref-s2-2-3-3"

TAILCUT_LABEL = "Tailcut-5-10-kill"

PART = 0         # 0 for gamma, 1 for protons

FAINT_BRIGHT_BORDER = 100

full_df = pd.read_csv(CSV_FILE_PATH)

ref = full_df[full_df.Type == 'Ref']
ref = ref[ref.Part == PART]

tc = full_df[full_df.Type == TAILCUT_LABEL]
tc = tc[tc.Part == PART]

wt = full_df[full_df.Type == WAVELET_LABEL]
wt = wt[wt.Part == PART]

tc_wt = pd.merge(tc, wt, on="Id", how="outer", suffixes=('_tc', '_wt'))  #.dropna(how='any')
df = pd.merge(tc_wt, ref, on="Id", how="outer")

df["delta_tc"] = np.fmod(((df['hPsi'] - df['hPsi_tc']) * 180. / np.pi), 90.)
df["delta_wt"] = np.fmod(((df['hPsi'] - df['hPsi_wt']) * 180. / np.pi), 90.)

df.delta_tc = abs(df.delta_tc)
df.delta_wt = abs(df.delta_wt)

df = df[df.border > 0]   # CONTAINED
#df = df[df.border == 0]  # NOT CONTAINED

fig = plot_delta_psi(df)

fig.suptitle(WAVELET_LABEL + " / " + TAILCUT_LABEL + " CONTAINED", fontsize=20)

In [ ]:
CSV_FILE_PATH = "../xps/best.csv"
#CSV_FILE_PATH = "../xps/2017_02_20/2017_02_20.csv"

#WAVELET_LABEL = "Wavelets-n4-K-k-C1-s3-m3-kill"
#WAVELET_LABEL = "WT-ref-f3-s3"
#WAVELET_LABEL = "WT-t24-f3-s4"
#WAVELET_LABEL = "WT-t28-f3-s5"
WAVELET_LABEL = "WT-ref-s2-2-3-3"

TAILCUT_LABEL = "Tailcut-5-10-kill"

PART = 0         # 0 for gamma, 1 for protons

FAINT_BRIGHT_BORDER = 100

full_df = pd.read_csv(CSV_FILE_PATH)

ref = full_df[full_df.Type == 'Ref']
ref = ref[ref.Part == PART]

tc = full_df[full_df.Type == TAILCUT_LABEL]
tc = tc[tc.Part == PART]

wt = full_df[full_df.Type == WAVELET_LABEL]
wt = wt[wt.Part == PART]

tc_wt = pd.merge(tc, wt, on="Id", how="outer", suffixes=('_tc', '_wt'))  #.dropna(how='any')
df = pd.merge(tc_wt, ref, on="Id", how="outer")

df["delta_tc"] = np.fmod(((df['hPsi'] - df['hPsi_tc']) * 180. / np.pi), 90.)
df["delta_wt"] = np.fmod(((df['hPsi'] - df['hPsi_wt']) * 180. / np.pi), 90.)

df.delta_tc = abs(df.delta_tc)
df.delta_wt = abs(df.delta_wt)

#df = df[df.border > 0]   # CONTAINED
df = df[df.border == 0]  # NOT CONTAINED

fig = plot_delta_psi(df)

fig.suptitle(WAVELET_LABEL + " / " + TAILCUT_LABEL + " NOT CONTAINED", fontsize=20)

## Multiplicity

See http://localhost:8888/notebooks/delta_psi_analysis_with_multiplicity.ipynb#Ratio